# Code for solving the Wumpus World problem

In [1]:
import numpy as np

In [2]:
# Helper function to get the next state
def get_next_state(s, action, grid_size):
    if action == 'UP':
        # If the agent tries to move up from the top row, stay in the same state
        return max(s - grid_size, 0) if s >= grid_size else s
    elif action == 'DOWN':
        # If the agent tries to move down from the bottom row, stay in the same state
        return min(s + grid_size, grid_size**2 - 1) if s < grid_size*(grid_size - 1) else s
    elif action == 'LEFT':
        # If the agent tries to move left from the first column, stay in the same state
        return s if s % grid_size == 0 else s - 1
    elif action == 'RIGHT':
        # If the agent tries to move right from the last column, stay in the same state
        return s if (s + 1) % grid_size == 0 else s + 1
    return s

In [144]:
# Value Iteration Function, Students need to implement the following loop
def MDP_value_iteration(S, A, P, R, gamma, eta, max_iter):
    # Input: S, A, P, R, gamma, eta, max_iter
    # S: set of states, stored as a list of integers
    # A: set of actions, stored as a list of strings, e.g. ['UP', 'DOWN', 'LEFT', 'RIGHT']
    # P: transition probabilities matrix, stored as a 3D numpy array, P[s',s,a] = P(s'|s,a)
    # R: reward function, stored as a 1D numpy array, R[s] = R(s)
    # gamma: discount factor
    # eta: convergence factor
    # max_iter: maximum number of iterations
    # Initialize the utilities for each state as zeros
    U = np.zeros(len(S))
    # print(A)
    # print(S)
    # print(P)
    # print(R) R is list for each tile its reward [-0.4, 10, -10, ...]
    # print(type(U)) U is np array with 16 zero

    for i in range(max_iter):
        # Create a copy of the current utility values
        U_prev = U.copy()
        # print(U_prev)
        # Iterate over all states in S
        for s in S:
            # TODO: Update the utility U[s]
            # Hints:
            # 1. Calculate the sum of utilities for each action a in A
            # 2. Use the Bellman equation: R[s] + gamma * max(sum(P(s'|s,a) * U_prev[s']) for each a in A)
            # 3. P(s'|s,a) is the probability of transitioning to state s' from state s given action a
            # 4. U_prev[s'] is the utility of the state s' from the previous iteration
            max_utility = float("-inf")
            for a in A:
                transition_sum = 0.0
                for s_prime in P[s][a]:
                    transition_prob = P[s][a][s_prime] # P(s'|s,a) 
                    transition_sum += transition_prob * U_prev[s_prime] # sum(P(s'|s,a) * U_prev[s']
                utility = R[s] + gamma * transition_sum # R[s] + gamma * max(sum(P(s'|s,a) * U_prev[s'])
                max_utility = max(max_utility, utility)
            U[s] = max_utility
        # if i < 5:
        #     print(U)

                
        # Check for convergence
        # TODO: Break the loop if the maximum change in utility values across all states is less than eta
        # Hints:
        # 1. Find the maximum absolute change in utilities
        # 2. Compare this value with eta

        # Check for convergence
        max_change = np.max(np.abs(U - U_prev))
        if max_change < eta:
            break

    # Return the final utilities
    # print(U)
    return U

# Policy Generation Function
def MDP_policy(S, A, P, U):
    # policy[s] is the best action to take in state s, firstly set it to 0 for all states
    policy = np.zeros(len(S), dtype=int)
    # Iterate over all states in S
    
    for s in S:
        which_is_better = []
        # TODO: Update the policy for the current state s
        # Hints: Given the current utility values U, read off the best action to take in state s
        # print(U)     
        for a in A:
            transitional_prob = 0
            for s_prime in P[s][a]:
                transitional_prob += P[s][a][s_prime] * U[s_prime] # P(s'|s,a) 
            which_is_better.append(transitional_prob)
        which_is_better = np.asarray(which_is_better)
        policy[s] = np.argmax(which_is_better)
            
                
            

        
    return policy

In [149]:
# Define the Wumpus world
grid_size = 4  # 4x4 grid
S = range(grid_size**2)  # States
A = ['RIGHT', 'LEFT', 'DOWN', 'UP']  # Actions

# Define the transition probabilities with stochastic movement
# P[s][a][s'] = P(s'|s,a)
P = {s: {a: {} for a in A} for s in S}
for s in S:
    for a in A:
        intended_state = get_next_state(s, a, grid_size)
        P[s][a][intended_state] = 0.8
        if a in ['LEFT', 'RIGHT']:
            P[s][a][get_next_state(s, 'UP', grid_size)] = 0.1
            P[s][a][get_next_state(s, 'DOWN', grid_size)] = 0.1
        else:
            P[s][a][get_next_state(s, 'LEFT', grid_size)] = 0.1
            P[s][a][get_next_state(s, 'RIGHT', grid_size)] = 0.1

# Define the rewards for each state
R = [-0.4] * 16
R[3] = 10   # Gold
R[10] = -5  # Pit
R[14] = -5  # Pit
R[13] = -10 # Wumpus

# Run value iteration
gamma = 0.9
eta = 0.1
max_iter = 10000
U = MDP_value_iteration(S, A, P, R, gamma, eta, max_iter)

# Policy representation for printing
policy_repr = {0: '→', 1: '←', 2: '↓', 3: '↑'} 

# Generate policy
policy = MDP_policy(S, A, P, U)

# Print utilities and policy in a 4x4 grid
print("Utilities and Policy for the Given Wumpus World:")
for i in range(grid_size):
    for j in range(grid_size):
        state = i * grid_size + j
        print(f"{U[state]:.2f} {policy_repr[policy[state]]}", end=" | ")
    print()

Utilities and Policy for the Given Wumpus World:
27.79 → | 32.73 → | 38.47 → | 45.14 ← | 
24.50 → | 28.57 → | 33.25 ↑ | 38.47 ↑ | 
21.24 ↑ | 24.15 ↑ | 23.93 ↑ | 32.27 ↑ | 
17.31 ↑ | 10.48 ↑ | 17.97 → | 26.78 ↑ | 
